<a href="https://colab.research.google.com/github/JakeAMystery/FORE_ClassWork/blob/main/BDDA_Session_5_missing_values_pipelining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Last amended: 1st June, 2023
# Ref: https://www.kaggle.com/code/alexisbcook/missing-values
# Data Source: https://www.kaggle.com/competitions/tabular-playground-series-aug-2022/data


This [data represents](https://www.kaggle.com/competitions/tabular-playground-series-aug-2022/data) the results of a large product testing study. For each product_code you are given a number of product attributes (fixed for the code) as well as a number of measurement values for each individual product, representing various lab testing methods. Each product is used in a simulated real-world environment experiment, and and absorbs a certain amount of fluid (loading) to see whether or not it fails.

Your task is to use the data to predict individual product failures of new codes with their individual lab test results.

# Simple Data pipelining

In [50]:
# 1.0

import pandas as pd
from random import randint

# 1.0.1
from sklearn.model_selection import train_test_split
from sklearn.ensemble import  RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA

# 1.0.2
from pathlib import Path


In [2]:
# 1.1
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [3]:
# 1.2 Display from a cell outputs of multiple commands:

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [4]:
# 1.2.1 Where is our data:

pathToFolder = "/gdrive/MyDrive/bdda1/"


In [5]:
# 1.2.2 And our file?

path = Path(pathToFolder) / "producttesting.zip"

In [6]:
# 2.1 Load the data
data = pd.read_csv(path)

In [7]:
# 2.2
data.shape  #(26570, 26)
data.head()


(26570, 26)

,id,product_code,loading,attribute_0,attribute_1,attribute_2,attribute_3,measurement_0,measurement_1,measurement_2,...,measurement_9,measurement_10,measurement_11,measurement_12,measurement_13,measurement_14,measurement_15,measurement_16,measurement_17,failure
0,0,A,80.10,material_7,material_8,9,5,7,8,4,...,10.672,15.859,17.594,15.193,15.029,NaN,13.034,14.684,764.100,0
1,1,A,84.89,material_7,material_8,9,5,14,3,3,...,12.448,17.947,17.915,11.755,14.732,15.425,14.395,15.631,682.057,0
2,2,A,82.43,material_7,material_8,9,5,12,1,5,...,12.715,15.607,NaN,13.798,16.711,18.631,14.094,17.946,663.376,0
3,3,A,101.07,material_7,material_8,9,5,13,2,6,...,12.471,16.346,18.377,10.020,15.250,15.562,16.154,17.172,826.282,0
4,4,A,188.06,material_7,material_8,9,5,9,2,8,...,10.337,17.082,19.932,12.428,16.182,12.760,13.153,16.412,579.885,0


In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26570 entries, 0 to 26569
Data columns (total 26 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              26570 non-null  int64  
 1   product_code    26570 non-null  object 
 2   loading         26320 non-null  float64
 3   attribute_0     26570 non-null  object 
 4   attribute_1     26570 non-null  object 
 5   attribute_2     26570 non-null  int64  
 6   attribute_3     26570 non-null  int64  
 7   measurement_0   26570 non-null  int64  
 8   measurement_1   26570 non-null  int64  
 9   measurement_2   26570 non-null  int64  
 10  measurement_3   26189 non-null  float64
 11  measurement_4   26032 non-null  float64
 12  measurement_5   25894 non-null  float64
 13  measurement_6   25774 non-null  float64
 14  measurement_7   25633 non-null  float64
 15  measurement_8   25522 non-null  float64
 16  measurement_9   25343 non-null  float64
 17  measurement_10  25270 non-null 

In [9]:
data.isna().sum()

id                   0
product_code         0
loading            250
attribute_0          0
attribute_1          0
attribute_2          0
attribute_3          0
measurement_0        0
measurement_1        0
measurement_2        0
measurement_3      381
measurement_4      538
measurement_5      676
measurement_6      796
measurement_7      937
measurement_8     1048
measurement_9     1227
measurement_10    1300
measurement_11    1468
measurement_12    1601
measurement_13    1774
measurement_14    1874
measurement_15    2009
measurement_16    2110
measurement_17    2284
failure              0
dtype: int64

In [10]:
data.corr()

<ipython-input-10-c44ded798807>:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  data.corr()


,id,loading,attribute_2,attribute_3,measurement_0,measurement_1,measurement_2,measurement_3,measurement_4,measurement_5,...,measurement_9,measurement_10,measurement_11,measurement_12,measurement_13,measurement_14,measurement_15,measurement_16,measurement_17,failure
id,1.000000,0.004305,-0.741200,0.558881,-0.253626,0.400206,0.000128,-0.005923,0.011313,-0.007001,...,0.001550,-0.029611,0.009524,-0.151220,-0.037559,-0.053744,0.083951,-0.059849,-0.000983,-0.007545
loading,0.004305,1.000000,-0.008208,-0.000756,-0.005578,0.002532,0.000303,-0.005203,-0.011792,-0.005599,...,-0.007751,-0.007525,0.003594,-0.002407,0.006138,0.000120,-0.008087,-0.000633,0.000305,0.129089
attribute_2,-0.741200,-0.008208,1.000000,-0.538697,0.175387,-0.377971,0.030522,0.005849,-0.011797,0.006811,...,-0.000098,-0.004953,0.094038,0.226756,0.049639,0.020921,-0.161242,0.035795,0.004729,0.006337
attribute_3,0.558881,-0.000756,-0.538697,1.000000,-0.370232,0.329233,0.013258,-0.002265,0.015470,-0.004034,...,-0.002295,-0.100845,-0.025434,-0.132060,0.028472,-0.049711,0.187519,-0.061520,-0.001418,-0.019222
measurement_0,-0.253626,-0.005578,0.175387,-0.370232,1.000000,-0.255255,-0.083813,-0.004890,-0.006459,0.003304,...,0.009112,0.032779,0.001878,0.033496,-0.031527,0.021907,-0.055431,0.003025,0.003351,0.009646
measurement_1,0.400206,0.002532,-0.377971,0.329233,-0.255255,1.000000,0.005032,-0.008363,0.001535,-0.005587,...,0.004529,0.011986,-0.035312,-0.084605,0.007523,-0.011471,0.060870,-0.004509,-0.004651,-0.010810
measurement_2,0.000128,0.000303,0.030522,0.013258,-0.083813,0.005032,1.000000,-0.002751,-0.009797,0.000241,...,0.001884,0.018695,-0.003877,-0.001808,0.029145,0.014623,0.000138,0.033088,-0.009596,0.015808
measurement_3,-0.005923,-0.005203,0.005849,-0.002265,-0.004890,-0.008363,-0.002751,1.000000,0.009573,0.009864,...,0.008195,-0.004854,-0.004427,0.002445,0.005893,0.002150,-0.010986,-0.002164,0.080902,0.003577
measurement_4,0.011313,-0.011792,-0.011797,0.015470,-0.006459,0.001535,-0.009797,0.009573,1.000000,0.005113,...,0.009080,-0.012998,-0.002784,-0.002586,-0.002454,-0.001713,-0.004567,0.002929,0.193045,-0.010488
measurement_5,-0.007001,-0.005599,0.006811,-0.004034,0.003304,-0.005587,0.000241,0.009864,0.005113,1.000000,...,-0.003589,0.002170,0.005306,0.000858,-0.009325,0.008720,-0.009101,-0.002996,0.450024,0.018079


In [11]:
# 2.2.1 Get target
y = data.pop('failure')

In [12]:
X = data.select_dtypes(exclude = ['object'])
X.pop("id")

0            0
1            1
2            2
3            3
4            4
         ...  
26565    26565
26566    26566
26567    26567
26568    26568
26569    26569
Name: id, Length: 26570, dtype: int64

In [13]:
X.head()

,loading,attribute_2,attribute_3,measurement_0,measurement_1,measurement_2,measurement_3,measurement_4,measurement_5,measurement_6,...,measurement_8,measurement_9,measurement_10,measurement_11,measurement_12,measurement_13,measurement_14,measurement_15,measurement_16,measurement_17
0,80.10,9,5,7,8,4,18.040,12.518,15.748,19.292,...,20.155,10.672,15.859,17.594,15.193,15.029,NaN,13.034,14.684,764.100
1,84.89,9,5,14,3,3,18.213,11.540,17.717,17.893,...,17.889,12.448,17.947,17.915,11.755,14.732,15.425,14.395,15.631,682.057
2,82.43,9,5,12,1,5,18.057,11.652,16.738,18.240,...,18.288,12.715,15.607,NaN,13.798,16.711,18.631,14.094,17.946,663.376
3,101.07,9,5,13,2,6,17.295,11.188,18.576,18.339,...,19.060,12.471,16.346,18.377,10.020,15.250,15.562,16.154,17.172,826.282
4,188.06,9,5,9,2,8,19.346,12.950,16.990,15.746,...,18.093,10.337,17.082,19.932,12.428,16.182,12.760,13.153,16.412,579.885


In [14]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26570 entries, 0 to 26569
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   loading         26320 non-null  float64
 1   attribute_2     26570 non-null  int64  
 2   attribute_3     26570 non-null  int64  
 3   measurement_0   26570 non-null  int64  
 4   measurement_1   26570 non-null  int64  
 5   measurement_2   26570 non-null  int64  
 6   measurement_3   26189 non-null  float64
 7   measurement_4   26032 non-null  float64
 8   measurement_5   25894 non-null  float64
 9   measurement_6   25774 non-null  float64
 10  measurement_7   25633 non-null  float64
 11  measurement_8   25522 non-null  float64
 12  measurement_9   25343 non-null  float64
 13  measurement_10  25270 non-null  float64
 14  measurement_11  25102 non-null  float64
 15  measurement_12  24969 non-null  float64
 16  measurement_13  24796 non-null  float64
 17  measurement_14  24696 non-null 

In [42]:
# 3.0 Divide data into training and validation subsets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [16]:
columns = X.columns.tolist()
fitter = SimpleImputer(strategy = 'median').fit(X_train)
X_train = fitter.transform(X_train); X_test = fitter.transform(X_test)

In [22]:
type(X); type(X.iloc[:,0]); type(X.iloc[0,0]); X.shape

X = pd.DataFrame(X, columns=columns); X

pandas.core.frame.DataFrame

pandas.core.series.Series

numpy.float64

(26570, 21)

,loading,attribute_2,attribute_3,measurement_0,measurement_1,measurement_2,measurement_3,measurement_4,measurement_5,measurement_6,...,measurement_8,measurement_9,measurement_10,measurement_11,measurement_12,measurement_13,measurement_14,measurement_15,measurement_16,measurement_17
0,80.10,9,5,7,8,4,18.040,12.518,15.748,19.292,...,20.155,10.672,15.859,17.594,15.193,15.029,NaN,13.034,14.684,764.100
1,84.89,9,5,14,3,3,18.213,11.540,17.717,17.893,...,17.889,12.448,17.947,17.915,11.755,14.732,15.425,14.395,15.631,682.057
2,82.43,9,5,12,1,5,18.057,11.652,16.738,18.240,...,18.288,12.715,15.607,NaN,13.798,16.711,18.631,14.094,17.946,663.376
3,101.07,9,5,13,2,6,17.295,11.188,18.576,18.339,...,19.060,12.471,16.346,18.377,10.020,15.250,15.562,16.154,17.172,826.282
4,188.06,9,5,9,2,8,19.346,12.950,16.990,15.746,...,18.093,10.337,17.082,19.932,12.428,16.182,12.760,13.153,16.412,579.885
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26565,158.95,6,9,6,16,4,16.301,13.259,18.068,15.505,...,19.354,NaN,12.177,17.942,10.112,15.795,18.572,16.144,NaN,729.131
26566,146.02,6,9,10,12,8,17.543,NaN,17.984,19.078,...,19.563,11.242,14.179,20.564,10.234,14.450,14.322,13.146,16.471,853.924
26567,115.62,6,9,1,10,1,15.670,11.535,16.778,18.385,...,19.279,11.407,16.437,17.476,8.668,15.069,16.599,15.590,14.065,750.364
26568,106.38,6,9,2,9,4,18.059,NaN,16.918,18.101,...,19.358,11.392,17.064,17.814,14.928,16.273,15.485,13.624,12.865,730.156


In [24]:
X_train = pd.DataFrame(X_train, columns=columns); X_train

X_test = pd.DataFrame(X_test, columns=columns); X_test

,loading,attribute_2,attribute_3,measurement_0,measurement_1,measurement_2,measurement_3,measurement_4,measurement_5,measurement_6,...,measurement_8,measurement_9,measurement_10,measurement_11,measurement_12,measurement_13,measurement_14,measurement_15,measurement_16,measurement_17
0,107.53,9.0,5.0,16.0,2.0,4.0,17.787,11.714,16.814,17.490,...,19.361,11.316,16.426,18.014,14.962,16.743,14.487,14.351,15.737,705.885
1,128.99,6.0,6.0,3.0,9.0,5.0,17.242,11.003,18.827,18.099,...,18.963,10.382,15.601,18.873,11.384,15.002,16.029,19.237,16.437,841.809
2,128.33,6.0,9.0,4.0,16.0,5.0,16.094,12.303,15.482,17.219,...,19.338,11.776,18.614,19.696,10.480,15.294,16.715,15.292,15.861,632.835
3,125.21,6.0,6.0,11.0,9.0,6.0,16.677,12.402,17.491,16.756,...,19.947,10.667,15.419,21.130,13.625,15.095,15.394,12.846,19.156,643.322
4,106.12,5.0,8.0,5.0,6.0,6.0,16.963,11.773,15.789,17.519,...,18.009,11.512,16.129,17.362,10.523,15.199,17.476,14.973,15.472,545.206
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19922,122.62,5.0,8.0,6.0,4.0,0.0,16.833,10.100,17.987,16.285,...,20.492,10.759,15.043,18.688,11.021,14.296,16.029,17.679,14.227,701.398
19923,138.01,6.0,6.0,9.0,16.0,5.0,17.787,12.992,17.326,18.176,...,18.594,10.357,15.578,20.981,11.566,15.625,16.029,14.543,17.240,756.658
19924,103.08,8.0,8.0,7.0,11.0,6.0,18.015,13.619,17.877,18.363,...,21.002,12.123,14.643,18.219,12.342,17.057,16.029,14.448,17.957,874.878
19925,203.25,5.0,8.0,5.0,11.0,9.0,17.064,13.041,18.591,18.825,...,20.122,12.415,15.710,18.386,11.424,15.065,14.803,17.140,18.435,960.722


,loading,attribute_2,attribute_3,measurement_0,measurement_1,measurement_2,measurement_3,measurement_4,measurement_5,measurement_6,...,measurement_8,measurement_9,measurement_10,measurement_11,measurement_12,measurement_13,measurement_14,measurement_15,measurement_16,measurement_17
0,134.55,6.0,6.0,15.0,16.0,10.0,15.581,9.881,15.976,16.455,...,19.353,11.118,18.604,20.113,14.415,16.450,18.116,13.227,17.776,537.384
1,122.90,5.0,8.0,14.0,13.0,6.0,16.843,11.629,17.267,16.352,...,18.059,11.193,18.787,19.670,11.723,17.800,16.707,12.752,15.080,613.661
2,172.83,6.0,6.0,4.0,10.0,17.0,18.456,11.569,17.189,17.557,...,19.227,11.455,15.142,19.888,11.723,15.987,14.449,15.110,18.744,716.930
3,134.32,8.0,8.0,6.0,9.0,5.0,17.827,10.524,17.435,18.702,...,18.448,11.979,16.083,18.886,11.847,16.132,16.170,17.187,16.437,692.657
4,185.81,9.0,5.0,5.0,1.0,2.0,18.324,12.267,17.477,18.438,...,18.682,11.075,14.768,20.069,13.573,14.541,18.534,18.079,18.978,774.008
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6638,133.95,5.0,8.0,10.0,7.0,3.0,17.074,10.591,17.569,17.508,...,19.243,13.118,16.253,19.214,10.715,15.059,15.063,18.036,19.376,810.489
6639,143.39,6.0,6.0,10.0,7.0,11.0,18.021,11.868,17.770,15.909,...,19.800,11.505,13.432,17.766,11.655,15.991,15.109,17.038,14.410,701.338
6640,134.32,9.0,5.0,15.0,4.0,2.0,16.959,12.548,18.790,18.779,...,18.518,11.855,16.168,20.896,11.996,16.186,18.171,12.783,13.019,789.047
6641,133.87,6.0,6.0,4.0,13.0,9.0,15.388,12.215,16.167,18.930,...,19.078,12.049,17.165,20.039,12.363,15.599,17.237,15.095,17.883,746.466


In [25]:
X_train.isna().sum()
X_test.isna().sum()

loading           0
attribute_2       0
attribute_3       0
measurement_0     0
measurement_1     0
measurement_2     0
measurement_3     0
measurement_4     0
measurement_5     0
measurement_6     0
measurement_7     0
measurement_8     0
measurement_9     0
measurement_10    0
measurement_11    0
measurement_12    0
measurement_13    0
measurement_14    0
measurement_15    0
measurement_16    0
measurement_17    0
dtype: int64

loading           0
attribute_2       0
attribute_3       0
measurement_0     0
measurement_1     0
measurement_2     0
measurement_3     0
measurement_4     0
measurement_5     0
measurement_6     0
measurement_7     0
measurement_8     0
measurement_9     0
measurement_10    0
measurement_11    0
measurement_12    0
measurement_13    0
measurement_14    0
measurement_15    0
measurement_16    0
measurement_17    0
dtype: int64

In [27]:
scaler = MinMaxScaler().fit(X_train)
X_train = scaler.transform(X_train); X_test = scaler.transform(X_test)

In [28]:
model = RandomForestClassifier().fit(X_train, y_train)

In [29]:
predictions = model.predict(X_test)


0.7875959656781575

In [30]:
f'{accuracy_score(y_test, predictions)*100:.2f}%'

'78.76%'

In [ ]:
def Avg_accuracy_score(random_scores, X, y):
  scores = []
  for i in random_scores:
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                        random_state=i)
    model = RandomForestClassifier(random_state=i).fit(X_train, y_train)
    predictions = model.predict(X_valid)
    scores.append(accuracy_score(y_valid, predictions))
  return scores.mean()

In [ ]:
f'{Avg_accuracy_score(, X, y)*100:.2f}%'

In [ ]:
pipe = make_pipeline(SimpleImputer(strategy = 'mean'), RandomForestClassifier())

In [ ]:
pipe.fit(X_train,y_train)

Pipeline(steps=[('simpleimputer', SimpleImputer()),
                ('randomforestclassifier', RandomForestClassifier())])

In [ ]:
pred = pipe.predict(X_valid)

In [ ]:
accu = accuracy_score(y_valid,pred)
accu   # 79%

0.7922468949943545

# Your Turn

a. Add Standard Scaler and PCA to it  
b. Add cross validation to it

In [31]:
### ----------------------- Done -----------------

In [57]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [58]:
pipe = make_pipeline(SimpleImputer(strategy = 'median'), MinMaxScaler(), PCA(0.95), RandomForestClassifier())

In [59]:
pipe.fit(X_train, y_train)

Pipeline(steps=[('simpleimputer', SimpleImputer(strategy='median')),
                ('minmaxscaler', MinMaxScaler()),
                ('pca', PCA(n_components=0.95)),
                ('randomforestclassifier', RandomForestClassifier())])

In [60]:
pred = pipe.predict(X_test)

In [61]:
f'{accuracy_score(y_test, pred)*100:.2f}%'

'78.47%'

In [62]:
def Avg_accuracy_score(model, X, y, iterations):
  scores = []
  for i in iterations:
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                        random_state=i)
    predictions = model.predict(X_valid)
    scores.append(accuracy_score(y_valid, predictions))
  return sum(scores)/len(scores)

In [68]:
f'{Avg_accuracy_score(pipe, X, y, [randint(1,5000) for i in range(10)])*100:.2f}%'

'94.69%'